In [1]:
# Author: Dr James K Ruffle
# Email: j.ruffle@ucl.ac.uk

In [2]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
import numpy as np
from tqdm import tqdm
from haversine import haversine, Unit

import os
print(os.getcwd())

/Users/jruffle/Library/CloudStorage/OneDrive-UniversityCollegeLondon/PhD/scripts_JKR/WellcomeHackathon/WIdeathonRetention23/code/dashboard_visualisations


In [3]:
df = pd.read_csv('../simulated_data/simulated_data.csv',index_col=0)
geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
token='pk.eyJ1IjoianJ1ZmZsZSIsImEiOiJjbGh1bWFrZWswMThnM2hyd2RrdzNndnNxIn0.JAGL1RE_mid-InPjlvx4hQ'
px.set_mapbox_access_token(token)

/var/folders/sw/h3fq4qtj0jj9kh01vd62l8980000gn/T/ipykernel_74984/2708870602.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../simulated_data/simulated_data.csv',index_col=0)
/var/folders/sw/h3fq4qtj0jj9kh01vd62l8980000gn/T/ipykernel_74984/2708870602.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_cities' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))


In [4]:
# Create a function to check proximity and group individuals
def group_coordinates(df, distance):
    groups = []
    visited = set()

    for i in range(len(df)):
        if i not in visited:
            group = [i]
            visited.add(i)

            for j in range(i + 1, len(df)):
                if j not in visited:
                    coord1 = (df['Latitude'].iloc[i], df['Longitude'].iloc[i])
                    coord2 = (df['Latitude'].iloc[j], df['Longitude'].iloc[j])
                    if haversine(coord1, coord2, unit=Unit.KILOMETERS) <= distance:
                        group.append(j)
                        visited.add(j)

            groups.append(group)

    return groups

# Group individuals within Xkm of each other
distance_threshold = 50
groups = group_coordinates(df, distance_threshold)

In [5]:
grouped_individuals = pd.DataFrame(np.zeros(shape=(len(groups),3)),columns=['Latitude','Longitude','Patients'])

for i, row in tqdm(grouped_individuals.iterrows(),total=grouped_individuals.shape[0]):
  
    density = len(groups[i])
    
    lat =0
    long=0
    
    for d in range(density):
        lat = lat + df.loc[groups[i][d],'Latitude']
        long = long + df.loc[groups[i][d],'Longitude']
    
    lat /= density
    long /=density
    
    grouped_individuals.loc[i,'Latitude']=lat
    grouped_individuals.loc[i,'Longitude']=long
    grouped_individuals.loc[i,'Patients']=density

100%|█████████████████████████████████████████| 19/19 [00:00<00:00, 5359.59it/s]


In [6]:
fig = px.density_mapbox(grouped_individuals, lat='Latitude', lon='Longitude', 
                        z='Patients',
                        radius=40,
                        zoom=5,
#                         center=dict(lat=54, lon=-3),
                        mapbox_style="satellite-streets",
#                         mapbox_style='outdoors',
#                         mapbox_style='carto-positron',
                       title='Geospatial distribution',
                        range_color=(0,grouped_individuals['Patients'].max()/1.5)
                       )

fig.write_html('map_satellite_street.html')
# fig.show()

In [7]:
fig = px.density_mapbox(grouped_individuals, lat='Latitude', lon='Longitude', 
                        z='Patients',
                        radius=40,
                        zoom=5,
#                         center=dict(lat=54, lon=-3),
#                         mapbox_style="satellite-streets",
                        mapbox_style='outdoors',
#                         mapbox_style='carto-positron',
                       title='Geospatial distribution',
                        range_color=(0,grouped_individuals['Patients'].max()/1.5)
                       )

fig.write_html('map_outdoors.html')
# fig.show()

In [8]:
fig = px.density_mapbox(grouped_individuals, lat='Latitude', lon='Longitude', 
                        z='Patients',
                        radius=40,
                        zoom=5,
#                         center=dict(lat=54, lon=-3),
#                         mapbox_style="satellite-streets",
#                         mapbox_style='outdoors',
                        mapbox_style='carto-positron',
                       title='Geospatial distribution',
                        range_color=(0,grouped_individuals['Patients'].max()/1.5)
                       )

fig.write_html('map_carto-positron.html')
# fig.show()